In [7]:
%run config.ipynb
from collections import defaultdict
import math

from surprise import BaselineOnly, SVD, NormalPredictor
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

In [2]:
df = pd.read_csv("data/yelp_academic_dataset_sample005_filter.csv")
df.head(1)

,review_id,user_id,business_id,review_stars,review_text,review_text_after_cleaning,user_review_count,user_elite,user_friends,user_fans,user_average_stars,user_total_compliments,business_name,business_categories,business_stars,to_recommend,num_user_friends
0,yNB39szX3M8mTEzTtsgoCw,Y1iCYGvLf4ifPoXlKLGq-w,o2Qh4SiGYJ7BK4hP7dfkrw,5,This is an amazing indian Bistro!!I If I do sa...,amaz bistroi say myself never cuisin glad expe...,1,0,None,0,5.0,0,Saffron Indian Bistro,"Restaurants, Indian",4.5,True,0


# SGD Baseline

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["user_id", "business_id", "review_stars"]], reader)

In [4]:
full_trainset = data.build_full_trainset()

In [5]:
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)

In [6]:
algo.fit(full_trainset)

Estimating biases using sgd...


In [21]:
trainset, testset = train_test_split(data, test_size=.25, random_state=42)
algo.fit(trainset)

print("\nComputing recommendations...")
predictions = algo.test(testset)

print("\nEvaluating accuracy of model...")
print("RMSE: ", accuracy.mae(predictions, verbose=False))
print("MAE: ", accuracy.rmse(predictions, verbose=False))
print("ACC: ", macro_accuracy([i[2] for i in testset], [predictions[i].r_ui for i in range(len(predictions))]))

Estimating biases using sgd...

Computing recommendations...

Evaluating accuracy of model...
RMSE:  1.279616191694076
MAE:  1.4861991307195033
ACC:  1.0


In [27]:
def genres_sim(business1, business2, genres):
    genres1 = genres[business1]
    genres2 = genres[business2]
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(genres1)):
        x = genres1[i]
        y = genres2[i]
        sumxx += x * x
        sumyy += y * y
        sumxy += x * y

    return sumxy/math.sqrt(sumxx*sumyy)

genres_sim("ZRLmQ3oAuspAkk5R6BOfyg", "VyVIneSU7XAWgMBllI6LnQ", genres)

0.6761234037828132

In [26]:
df.iloc[4535]["business_id"]

'ZRLmQ3oAuspAkk5R6BOfyg'

In [21]:
df.iloc[20000]["business_id"]

'tIX6o1jkLrrmsJIdGUjA5A'

In [ ]:
algo.